In [1]:
import pandas as pd
import os
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#Load Data and review content
df = pd.read_csv("root_cause_analysis.csv")
#print("\nLoaded Data :\n------------------------------------")
print(df.dtypes)
df.head()





ID                   int64
CPU_LOAD             int64
MEMORY_LEAK_LOAD     int64
DELAY                int64
ERROR_1000           int64
ERROR_1001           int64
ERROR_1002           int64
ERROR_1003           int64
ROOT_CAUSE          object
dtype: object


,ID,CPU_LOAD,MEMORY_LEAK_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003,ROOT_CAUSE
0,1,0,0,0,0,1,0,1,MEMORY_LEAK
1,2,0,0,0,0,0,0,1,MEMORY_LEAK
2,3,0,1,1,0,0,1,1,MEMORY_LEAK
3,4,0,1,0,1,1,0,1,MEMORY_LEAK
4,5,1,1,0,1,0,1,0,NETWORK_DELAY


Let's convert the inputdata so that the machine learning algorithm can read easily

In [3]:
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
label_encoder = preprocessing.LabelEncoder()
df['ROOT_CAUSE'] = label_encoder.fit_transform(
                                df['ROOT_CAUSE'])

#Convert Pandas DataFrame to a numpy vector
np_df = df.to_numpy().astype(float)
print(np_df)

[[   1.    0.    0. ...    0.    1.    1.]
 [   2.    0.    0. ...    0.    1.    1.]
 [   3.    0.    1. ...    1.    1.    1.]
 ...
 [ 998.    1.    1. ...    0.    0.    1.]
 [ 999.    0.    1. ...    0.    0.    2.]
 [1000.    1.    0. ...    1.    0.    0.]]


In [5]:

#Extract the feature variables (X)
X_data = np_df[:,1:8] # we have 8 features 
#Extract the target variable (Y), conver to one-hot-encodign
Y_data=np_df[:,8] #making a separate column for each category and using 1s and 0s 
                  #to show which category each item belongs to
Y_data = tf.keras.utils.to_categorical(Y_data,3)    

#Split training and test data
X_train,X_test,Y_train,Y_test = train_test_split( X_data, Y_data, test_size=0.10)

print("Shape of feature variables :", X_train.shape)
print("Shape of target variable :",Y_train.shape)

Shape of feature variables : (900, 7)
Shape of target variable : (900, 3)


Now, Lets Build and Evaluate the data 
Building the Model: This involves designing the model's architecture (like layers and neurons) and configuring it (like setting the loss function and optimizer). This step is crucial to tailor the model to the
specific characteristics and requirements of the data and task at hand.
Evaluate the Model: After training you evaluate the data with the different data set to acess it performance

In [6]:
from tensorflow import keras
# Keras API from TensorFlow, a high-level neural networks library, for building and training deep learning models
from tensorflow.keras import optimizers
#optimization algorithms like SGD, Adam, etc., which are methods used to change the attributes of the neural 
#network such as weights and learning rate in order to reduce losses.
from tensorflow.keras.regularizers import l2
#which helps prevent overfitting by penalizing large weights in the model's learning process


#Setup Training Parameters
EPOCHS=20 ## The entire dataset will pass through the network in 20 times

BATCH_SIZE=64 #Determines how many data points are processed before the model's internal parameters are updated

VERBOSE=1 #Controls the level of detail shown during the training process.
#A setting of 1 displays a detailed progress bar and metrics for each epoch.

#ensuring the model's output layer has the correct number of neurons to match the number of distinct categories in the data
OUTPUT_CLASSES=len(label_encoder.classes_) 
N_HIDDEN=128
VALIDATION_SPLIT=0.2
#Create a Keras sequential model
model = tf.keras.models.Sequential()
#Add a Dense Layer
model.add(keras.layers.Dense(N_HIDDEN,
                             input_shape=(7,),
                              name='Dense-Layer-1',
                              activation='relu'))

#Add a second dense layer
model.add(keras.layers.Dense(N_HIDDEN,
                              name='Dense-Layer-2',
                              activation='relu'))

#Add a softmax layer for categorial prediction
model.add(keras.layers.Dense(OUTPUT_CLASSES,
                             name='Final',
                             activation='softmax'))

#Compile the model
model.compile(
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.summary()

#Build the model
model.fit(X_train,
          Y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=VERBOSE,
          validation_split=VALIDATION_SPLIT)


#Evaluate the model against the test dataset and print results
print("\nEvaluation against Test Dataset :\n------------------------------------")
model.evaluate(X_test,Y_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Dense-Layer-1 (Dense)       (None, 128)               1024      
                                                                 
 Dense-Layer-2 (Dense)       (None, 128)               16512     
                                                                 
 Final (Dense)               (None, 3)                 387       
                                                                 
Total params: 17923 (70.01 KB)
Trainable params: 17923 (70.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
12/12 [==============================] - 0s 10ms/step - loss: 0.9046 - accuracy: 0.7028 - val_loss: 0.6709 - val_accuracy: 0.8444
Epoch 2/20
12/12 [==============================] - 0s 3ms/step - loss: 0.6402 - accuracy: 0.8069 - val_loss: 0.4995 - val_accuracy: 0.8278
Epo

[0.3841954469680786, 0.8399999737739563]

Predicting the root cause 

In [12]:
#Pass individual flags to Predict the root cause
import numpy as np

CPU_LOAD=1 # Variable Initilization 
MEMORY_LOAD=0 #Model prediction 
DELAY=0
ERROR_1000=0
ERROR_1001=1
ERROR_1002=1
ERROR_1003=0

prediction=np.argmax(model.predict(
    [[CPU_LOAD,MEMORY_LOAD,DELAY,
      ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003]]), axis=1 )

print(label_encoder.inverse_transform(prediction))

1/1 [==============================] - 0s 23ms/step
[0]


In [8]:

print(label_encoder.inverse_transform(prediction))

[0]


In [ ]:
In machine learning models, especially in classification tasks, binary (0 and 1) encoding like this is commonly 
used to represent the presence or absence of a condition, the state of a system, categorical variables, and so on


In [9]:
#Predicting as a Batch
print(label_encoder.inverse_transform(np.argmax(
        model.predict([[1,0,0,0,1,1,0],
                                [0,1,1,1,0,0,0],
                                [1,1,0,1,1,0,1],
                                [0,0,0,0,0,1,0],
                                [1,0,1,0,1,1,1]]), axis=1 )))

1/1 [==============================] - 0s 23ms/step
[0 2 1 0 0]
